In [3]:
pip install -r install.txt

     |████████████████████████████████| 23.6MB 19.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


## Vegetation Index
### *Using NDVI (Normalized Difference Vegetation Index)

To calculate the NDVI, we need Red band and Near-Infrared Band (NIR). Different satellite images assign different numbers for these bands. Sentinel Images have red in 4th band and NIR in the 8th band. The formula for NDVI calculation is:

$$NDVI = \frac{nir - red}{nir + red} $$




In [9]:
import rasterio as rio
import numpy as np
from PIL import Image, ImageDraw
import glob
import os
import numpy as np

# The initial release contains only one tile, so lets hardcode its location
# here.  When you have more tiles, you can update this
TILE_X = 7680
TILE_Y = 10240

# The expected value of a Pixel in a mask file indicating that the pixel is
# within that region.  Tuple value, (Red, Green, Blue, Alpha)
IS_IN_MASK_PIXEL_VALUE = (0, 0, 0, 255)

# Tile width / height in pixels

TILE_WIDTH_PX = 512
TILE_HEIGHT_PX = 512

def get_timeseries_image_paths(tile_x, tile_y, band, date):
    path = f"./data/sentinel-2a-tile-{tile_x}x-{tile_y}y/timeseries/{tile_x}-{tile_y}-{band}-{date}.png"
    #path = glob.glob(path)
    #path = path[0] # get the first date 
    return path 

def get_red(tile_x, tile_y, date):
    band = 'B04'
    path = get_timeseries_image_paths(tile_x, tile_y, band , date)
    b4 = rio.open(path)
    red = b4.read()
    return red.astype(float)
    
def get_nir(tile_x, tile_y, date):
    band = 'B08'
    path = get_timeseries_image_paths(tile_x, tile_y, band, date)
    b8 = rio.open(path)
    nir = b8.read()
    return nir.astype(float)

def NDVI(tile_x, tile_y, date):
    red = get_red(tile_x, tile_y, date)
    nir = get_nir(tile_x, tile_y, date)
    return (nir-red)/(nir+red)


In [33]:
ndvi = NDVI(TILE_X, TILE_Y,'2016-12-22')
b4 = rio.open(get_timeseries_image_paths(TILE_X, TILE_Y, band = 'B04', date='2016-12-22'))
meta = b4.meta
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)

with rio.open('NDVI.tif', 'w', **meta) as dst:
    dst.write(ndvi.astype(rio.float32))
    
img = Image.open('NDVI.tif')
img.show()